# Advent of Code 2022

## Day 5: Supply Stacks

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

In [ ]:
from typing import TextIO, Iterator, Optional
from collections import deque

In [ ]:
# for reading the header containing the boxes - up to a blank line
def read_header(file: TextIO) -> Iterator[str]:
    for line in file:
        line = line.strip('\r\n')  # don't want to strip normal spaces in header!
        if not line.strip():
            break
        yield line

# for reading remaining lines containing moves - up to end (or max lines for testing/debugging)
def read_body(file: TextIO, limit: Optional[int] = None) -> Iterator[str]:
    count = 0
    for line in file:
        line = line.strip()
        yield line
        count += 1
        if limit is not None and count >= limit:
            break

In [ ]:
def parse_box_row(line: str) -> list[str]:
    # the line is 3 chars per column plus 1 space between each
    assert (len(line) + 1) % 4 == 0
    line = line + ' '
    rv = []
    for i in range(0, len(line), 4):
        rv.append(line[i+1].strip())
    return rv

In [ ]:
def build_header_to_stacks(file: TextIO) -> dict[int, deque]:
    deques = None
    for line in read_header(file):
        row = parse_box_row(line)
        if not deques:
            deques = [deque() for _ in row]
        assert (len(deques) == len(row)), 'non-uniform row lengths for header'
        for d, element in zip(deques, row):
            if element:
                # using appendleft because the txt file has items in oposite of stack order
                d.appendleft(element)
    index = 1
    rv = {}
    for d in deques:
        left = d.popleft()
        assert int(left) == index, 'expected left to deque to be a counter digit'
        rv[index] = d
        index += 1
    return rv

In [ ]:
class Move:

    __init__ = ['count', 'source', 'destination']

    def __init__(self, s: str):
        tokens = s.split(' ')
        assert len(tokens) == 6
        assert tokens[0] == 'move'
        self.count = int(tokens[1])
        assert tokens[2] == 'from'
        self.source = int(tokens[3])
        assert tokens[4] == 'to'
        self.destination = int(tokens[5])
        assert self.count >= 1
        assert self.source != self.destination

    def __str__(self):
        return f'move {self.count} from {self.source} to {self.destination}'

    def apply_to(self, stacks: dict[int, deque], move_method) -> None:
        source_stack = stacks[self.source]
        destination_stack = stacks[self.destination]
        assert (len(source_stack) >= self.count), f"can't move {self.count} form stack {source_stack}"
        move_method(source_stack, destination_stack, self.count)


def read_moves(file: TextIO, limit: Optional[int] = None) -> Iterator[Move]:
    for line in read_body(file, limit=limit):
        yield Move(line)

In [ ]:
def move_all(filename: str, move_method) -> str:

    with open(filename) as file:

        stacks = build_header_to_stacks(file)

        for move in read_moves(file):
            move.apply_to(stacks, move_method=move_method)

        print('The top of each stack after moving boxes is:', end=' ')
        for index in sorted(stacks):
            print(stacks[index].pop(), end='')
        print()

In [ ]:
INPUT_FILE = 'data/input05.txt'

### Part 1

In [ ]:
def move_one_at_a_time(source_stack: deque, destination_stack: deque, count: int) -> None:
    for _ in range(count):
        destination_stack.append(source_stack.pop())

In [ ]:
if __name__ == '__main__':
    move_all(INPUT_FILE, move_method=move_one_at_a_time)

### Part 2

In [ ]:
def move_at_once(source_stack: deque, destination_stack: deque, count: int) -> None:
    crane = deque()
    for _ in range(count):
        crane.append(source_stack.pop())
    while crane:
        destination_stack.append(crane.pop())

In [ ]:
if __name__ == '__main__':
    move_all(INPUT_FILE, move_method=move_at_once)